###CellPoseBatchSeg###

This script batch runs a pre-trained cell segmentation model from cellpose on a folder of images.
These are usually the larger images from which crops were taken to train the model.
Inputs: 
- segmentation model
- images with nuclei in channel 1 and cellbody marker in channel 2 (out of running ExtractForCellPose.ijm)
Outputs: 
- integer labelled cell segmentation masks. 

Instructions:
1. Ensure this jupyter notebook is running in the cellpose environment in anaconda (see 'installation below').
2. Ensure installation of imported packages (see 'installation below')
3. Specify directories at beginning of script. 
4. Flow Thresholds
4. Run on a a couple test images and (if desired) uncomment code under 'DISPLAY' to see the segmentation results. If processing many images best to leave this commented out to avoid jamming the image display in jupyter.
5. Enjoy time saved. :)

Installations:
- In anaconda terminal, install and open jupyter within cellpose environment:
    - `conda activate cellpose`
    - `conda install jupyter`
    - `pip install --upgrade requests jupyter`
    - 'jupyter notebook'
- In anaconda terminal (in activated cellpose environment), Install required modules:
    - `pip install chardet`
    - `pip install --upgrade charset-normalizer`
    - `conda install -c anaconda numpy`
    - `conda install -c conda-forge scikit-image`
    - `conda install -c conda-forge matplotlib`

script adapted from https://github.com/MouseLand/cellpose/blob/master/notebooks/run_cellpose_GPU.ipynb

author: heeva.baharlou@gmail.com (27/05/2023)

In [1]:
# Import libraries
import os
import skimage.io
from cellpose import models, core
from cellpose.io import logger_setup

Define your CellPose and any associated variables here:

In [3]:
# If using an IN-BUILT model, change the 'inbuilt_model' variable to the model name - otherwise, set to 'None'
inbuilt_model = "cyto3"
if inbuilt_model is not None:
    model = models.Cellpose(gpu=core.use_gpu(), model_type=inbuilt_model)

# If using a CUSTOM model, change the 'model_path' variable to the directory containing the model - otherwise, set to 'None'
custom_model_path = None
if custom_model_path is not None:
    model = models.CellposeModel(gpu=core.use_gpu(), pretrained_model=custom_model_path)

# Define any other variables you set when running your model
cell_diameter = 12.4
flow_threshold = 3
cellprob_threshold = -6
channels = [2, 1] # This means Channel 1 was G and Channel 2 was R (1 = R, 2 = G, 3 = B)

100%|██████████| 25.3M/25.3M [00:30<00:00, 884kB/s] 
100%|██████████| 3.54k/3.54k [00:00<00:00, 3.65MB/s]


Run the batch segmentation:

In [4]:
# Set and create directories
image_dir = "analysis/cellpose"
mask_dir = "analysis/segmentation_masks"
os.makedirs(mask_dir, exist_ok=True)
# Call logger_setup to have output of cellpose written
logger_setup()
# Get list of image files
files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".tif")]  # Adjust the file extension if necessary
imgs = [skimage.io.imread(f) for f in files]
# Run segmentation
masks, flows, styles, diams = model.eval(imgs, diameter=cell_diameter, flow_threshold=flow_threshold, cellprob_threshold=cellprob_threshold, channels=channels)
# Save mask images
for idx, mask in enumerate(masks):
    # Create a new file name by replacing the old directory with the new one
    file_name = files[idx].replace(image_dir, mask_dir)
    # Add a suffix to indicate it's a mask
    file_name = os.path.splitext(file_name)[0] + "_mask.tif"
    skimage.io.imsave(file_name, mask)

2024-04-04 10:51:06,546 [INFO] WRITING LOG OUTPUT TO C:\Users\oscar\.cellpose\run.log
2024-04-04 10:51:06,547 [INFO] 
cellpose version: 	3.0.7 
platform:       	win32 
python version: 	3.9.18 
torch version:  	1.8.2
2024-04-04 10:51:06,572 [INFO] channels set to [2, 1]
2024-04-04 10:51:06,573 [INFO] ~~~ FINDING MASKS ~~~
2024-04-04 10:53:30,643 [INFO] 100%|##########| 9/9 [02:24<00:00, 16.01s/it]
2024-04-04 10:53:30,644 [INFO] >>>> TOTAL TIME 144.07 sec


C:\Users\oscar\AppData\Local\Temp\ipykernel_12932\1220904177.py:18: UserWarning: analysis/segmentation_masks\20230922 LN 146 IFNghighconc negprobelowconc panel 6_s0_a1_ac_segment_CpSeg_mask.tif is a low contrast image
  skimage.io.imsave(file_name, mask)
C:\Users\oscar\AppData\Local\Temp\ipykernel_12932\1220904177.py:18: UserWarning: analysis/segmentation_masks\20230922 LN 146 IFNghighconc negprobelowconc panel 6_s0_a2_ac_segment_CpSeg_mask.tif is a low contrast image
  skimage.io.imsave(file_name, mask)
C:\Users\oscar\AppData\Local\Temp\ipykernel_12932\1220904177.py:18: UserWarning: analysis/segmentation_masks\20230922 LN 146 posprobehighconc_noprobelowconc_mw_panel 6_s0_a1_ac_segment_CpSeg_mask.tif is a low contrast image
  skimage.io.imsave(file_name, mask)
C:\Users\oscar\AppData\Local\Temp\ipykernel_12932\1220904177.py:18: UserWarning: analysis/segmentation_masks\20230922 LN 146 posprobehighconc_noprobelowconc_mw_panel 6_s0_a2_ac_segment_CpSeg_mask.tif is a low contrast image
  ski

Move selected images to a new 'for_cellprofiler' directory for use in CellProfiler later on:

In [1]:
# Import libraries
import shutil
from pathlib import Path

# Create 'for_cellprofiler' directory 
full_path = Path("analysis/full_images")
mask_path = Path("analysis/segmentation_masks")
dest_path = Path("analysis/for_cellprofiler")
dest_path.mkdir(exist_ok=True)
cellp_out = Path("analysis/CellProfilerOutput")
cellp_out.mkdir(exist_ok=True)
# Copy relevant images to new directory
for file in (list(full_path.glob("*.tiff")) + list(mask_path.glob("*.tif"))):
    shutil.copy(file, dest_path)